# Corpus

In [380]:
import pandas as pd                                     
import numpy as np                                      
import os 



import matplotlib.pyplot as plt

from datetime import datetime

%matplotlib inline
import seaborn as sns                                   # For pretty plots
import gensim
import nltk


from os import path
from wordcloud import WordCloud
from wordcloud import STOPWORDS
from PIL import Image
from gensim import corpora, models, similarities
from nltk.corpus import stopwords

In [381]:
emails = pd.read_csv("hillary-clinton-emails/Emails.csv")
emails.head()

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,...,ExtractedTo,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText
0,1,C05739545,WOW,H,"Sullivan, Jacob J",87.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739545...,F-2015-04841,...,NaN,"Sullivan, Jacob J <Sullivan11@state.gov>",NaN,"Wednesday, September 12, 2012 10:16 AM",F-2015-04841,C05739545,05/13/2015,RELEASE IN FULL,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
1,2,C05739546,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,H,NaN,NaN,2011-03-03T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739546...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739546,05/13/2015,RELEASE IN PART,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...",UNCLASSIFIED\nU.S. Department of State\nCase N...
2,3,C05739547,CHRIS STEVENS,;H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739547...,F-2015-04841,...,B6,"Mills, Cheryl D <MillsCD@state.gov>","Abedin, Huma","Wednesday, September 12, 2012 11:52 AM",F-2015-04841,C05739547,05/14/2015,RELEASE IN PART,Thx,UNCLASSIFIED\nU.S. Department of State\nCase N...
3,4,C05739550,CAIRO CONDEMNATION - FINAL,H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739550...,F-2015-04841,...,NaN,"Mills, Cheryl D <MillsCD@state.gov>","Mitchell, Andrew B","Wednesday, September 12,2012 12:44 PM",F-2015-04841,C05739550,05/13/2015,RELEASE IN PART,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
4,5,C05739554,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,"Abedin, Huma",H,80.0,2011-03-11T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739554...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739554,05/13/2015,RELEASE IN PART,"H <hrod17@clintonemail.com>\nFriday, March 11,...",B6\nUNCLASSIFIED\nU.S. Department of State\nCa...


In [382]:
emails_text = emails['ExtractedBodyText'].dropna()
#emails_text = emails_text.head(100)
#emails_text.shape
type(emails_text)

pandas.core.series.Series

In [383]:
emails_text = emails_text.apply(lambda x: x.lower())

In [384]:
emails_text = emails_text.apply(lambda x: x\
                                .replace(':',' ')\
                                .replace('—',' ')\
                                .replace('-',' ')\
                                .replace('.',' ')\
                                .replace(',',' ')\
                                .replace('.',' ')\
                                .replace('<',' ')\
                                .replace('>',' ')\
                                .replace('=',' ')\
                                .replace('•',' ')\
                                .replace("\\",' ')\
                                .replace('\n', ' ')
                                .replace('^',' ')\

                               )
#emails_text = emails_text.apply(lambda x: x.replace('—',' '))


In [385]:
type(emails_text)
#emails_text.iloc[0]
#emails_text

pandas.core.series.Series

In [386]:
#emails_text.head

In [387]:
documents = emails_text.tolist()
#documents

In [388]:
type(documents)
#documents

list

In [389]:
len(documents)

6742

In [416]:
# you must download the stoplist manually : type to pyhton
# console nltk.donwload()
'''stoplist = stopwords.words('english')
stoplist.append('pm')
stoplist.append('no')
stoplist.append('com')
stoplist.append('doc')
stoplist.append('docx')
stoplist.append('mr')
stoplist.append('mrs')

for i in range(1,100) :
    stoplist.append(i)
'''


stoplist = set(STOPWORDS)
stoplist.add('pls')
stoplist.add('yes')
stoplist.add('call')
stoplist.add('pm')
stoplist.add('no')
stoplist.add('com')
stoplist.add('doc')
stoplist.add('docx')
stoplist.add('pdf')
stoplist.add('mr')
stoplist.add('mrs')
stoplist.add('call')
stoplist.add('need')
stoplist.add('one')
stoplist.add('two')
for i in range(10,100) :
    stoplist.add(i)
print(stoplist)

{'are', 'itself', '', "she'll", 'two', 10, 11, 'most', "couldn't", 'an', 'after', 'its', 'been', 'ours', 'yourself', 12, 13, 14, 15, 16, 17, "don't", "you're", 'in', 20, 21, 22, 23, 24, 25, 26, 27, 28, 'would', 30, "he'll", 32, 33, 34, 35, 36, 37, 38, 39, 40, 'there', 42, 43, "she's", 45, 46, 47, 48, 49, 50, "he's", 'herself', "they'd", 'was', 55, 56, 57, "you'd", 59, 'docx', 61, 62, 63, 'both', 'could', 'no', 67, "mustn't", 69, 70, 71, 72, 73, 74, 'i', 'did', 77, 'over', "aren't", "shouldn't", 81, 18, 83, 84, 'doc', 'one', 19, 88, 89, 'ourselves', 91, 'such', 93, 94, 95, 96, "haven't", 98, 99, 'is', 'this', "shan't", "we're", "he'd", 'http', 'before', "it's", 'nor', 'our', 'very', 'theirs', 'pm', "that's", 'does', 'any', 'get', 29, "can't", 'doing', 'the', "you've", "we'd", 'and', "hasn't", 'hers', 'so', 31, "why's", 'have', 'off', 'them', 'having', 'again', "i'd", 'yours', "we'll", 'how', 'cannot', 'during', "you'll", 'yourselves', 'other', "wasn't", 41, "we've", 'that', 'each', 'tha

In [414]:
# remove list contains on stoplist and tokenize
texts = [[word for word in document.lower().split() if word not in stoplist and len(word) > 1]
         for document in documents]

In [392]:
len(texts)

6742

In [393]:
# counts the number of appartion of a word
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

In [394]:
len(frequency)

36907

In [395]:
# remove word that apprears only once
texts = [[token for token in text if frequency[token] > 1]
      for text in texts]

In [396]:
len(texts)

6742

In [397]:
#texts

In [398]:
dictionary = corpora.Dictionary(texts)
dictionary.save("part3/deerwester-email.dict")

In [399]:
print(dictionary)

Dictionary(18410 unique tokens: ['145', 'pouch', 'capabilities', 'rooms', 'mirage']...)


In [400]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [401]:
#print(corpus)

In [402]:
corpora.MmCorpus.serialize("part3/deerwester-email.mm", corpus)

In [403]:
dictionary = corpora.Dictionary.load("part3/deerwester-email.dict")


In [404]:
model = models.LdaModel(corpus, id2word=dictionary, num_topics=5)

In [405]:
model.print_topics(5)

[(0,
  '0.009*"will" + 0.005*"new" + 0.005*"fyi" + 0.005*"state" + 0.004*"one" + 0.004*"said" + 0.004*"also" + 0.004*"president" + 0.003*"people" + 0.003*"time"'),
 (1,
  '0.014*"2010" + 0.008*"will" + 0.007*"gov" + 0.007*"print" + 0.006*"pls" + 0.006*"pm" + 0.005*"re" + 0.005*"fw" + 0.004*"cheryl" + 0.004*"mills"'),
 (2,
  '0.005*"will" + 0.004*"one" + 0.004*"said" + 0.004*"government" + 0.003*"us" + 0.003*"obama" + 0.003*"new" + 0.003*"american" + 0.003*"president" + 0.003*"party"'),
 (3,
  '0.010*"will" + 0.007*"state" + 0.005*"also" + 0.005*"see" + 0.005*"now" + 0.004*"call" + 0.004*"want" + 0.004*"need" + 0.004*"tomorrow" + 0.004*"yes"'),
 (4,
  '0.031*"pm" + 0.014*"office" + 0.013*"30" + 0.012*"secretary\'s" + 0.011*"00" + 0.011*"call" + 0.010*"10" + 0.009*"will" + 0.009*"meeting" + 0.008*"15"')]

In [406]:
print(model)

LdaModel(num_terms=18410, num_topics=5, decay=0.5, chunksize=2000)


In [407]:
#print(corpus)